# 01 - Model Client Usage

Demonstrate various usage scenarion for Model Client.

## A - Text Generation

For most of these examples, the Qwen 3 model is used since it supports both reasoning/thinking and tool usage.

In [ ]:
from aimu.models import OllamaClient

model_client = OllamaClient(OllamaClient.MODELS.QWEN_3_8B)
print(model_client.model.name + "\n" + model_client.model.value)

For non-interactive (chat-based) generation, the *generate* method is used. This method allows you to specify a prompt and receive a generated response.

In [ ]:
model_client.generate("What is the capital of France?")

Output can be streamed as well.

In [ ]:
response = model_client.generate_streamed("What is the capital of England?")

for response_part in response:
    print(response_part, end="", flush=True)

Parameters can be passed to the generate (and chat) methods to control the output. For thinking/reasoning models, max_tokens should be large enough to capture both the reasoning output, which happens first, and the generation output.

In [ ]:
# For generate_kwargs, see https://github.com/ollama/ollama/blob/main/docs/modelfile.md#valid-parameters-and-values

model_client.generate(
    "What is the capital of Germany?",
    generate_kwargs={
        "temperature": 0.1,
        "top_p": 0.9,
        "top_k": 40,
        "max_tokens": 1024,
        "repeat_penalty": 1.1,
    },
)

## B - Basic Chat

Chatting with a model simply involves passing a message and receiving a response. The Model Client handles the conversation history automatically.

In [ ]:
model_client.chat("Where is the Caribbean?")

As with the generate method, the model response can be streamed as well. Content isn't streamed until thinking/reasoning models have completed their reasoning step, so for these models, there's a delay before the content is starts streaming.

In [ ]:
response = model_client.chat_streamed("Were there pirates there?")

for chunk in response:
    print(chunk, end="", flush=True)

The complete message history, including the model thinking/resoning, can be accessed via the `messages` property of the Model Client.

In [ ]:
model_client.messages

A custom system message can be set, or re-set, as desired.

In [ ]:
model_client.system_message = "You are a helpful assistant that responds using pirate speak."
model_client.messages[0]

In [ ]:
model_client.chat("Name a famous pirate from there.")

## C - Tool Usage

Create an MCP tool to use. See the "02 - MCP Tools" notebook for more examples of how to set up and use MCP tools.

In [ ]:
import datetime
from fastmcp import FastMCP

mcp = FastMCP("AIMU Tools")


@mcp.tool()
def get_current_date_and_time() -> str:
    """Returns the current data and time as a string."""
    return str(datetime.datetime.now())

Create a new AIMU MCPClient for handling MCP tool requests.

In [ ]:
from aimu.tools import MCPClient

# Required to allow nested event loops in Jupyter notebooks
import nest_asyncio

nest_asyncio.apply()

mcp_client = MCPClient(mcp)
model_client.mcp_client = mcp_client

In [ ]:
model_client.chat("What time is it?")

In [ ]:
model_client.messages

In [ ]:
response = model_client.chat_streamed("What time is it now?")

for chunk in response:
    print(chunk, end="", flush=True)

In [ ]:
model_client.messages